In [ ]:
## Standard libraries
import os
import math
import numpy as np 
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
# Set Seaborn style
sns.set(style='darkgrid', font_scale=1.2)
from sklearn.datasets import make_moons

## Progress bar
from tqdm.notebook import tqdm

import torch
print("Using torch", torch.__version__)
#torch.manual_seed(42) # Setting the seed
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

Using torch 1.13.1


In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    
    def forward(self, x):
        return self.layers(x)

class EBM(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    
    def forward(self, x):
        # Returns grad_z[ f_a(z) ]
        return self.layers(x)

In [ ]:
class PriorSampler():
    def __init__(self, K, s, device):
        self.device = device
        self.K = torch.tensor(K, device=self.device)
        self.s = torch.tensor(s, device=self.device)
    
    def get_sample(self, x, EBM):
        z_k = torch.randn_like(x, device=self.device)
        step=0
        while step < self.K:
            z_k = z_k + self.s * (EBM(z_k)-z_k) + (torch.sqrt(2*self.s)*torch.randn_like(z_k, device=self.device))
            step += 1
        
        return z_k

class PosteriorSamples():
    def __init__(self, K, s, device):
        self.device = device
        self.K = torch.tensor(K, device=self.device)
        self.s = torch.tensor(s, device=self.device)
    
    def get_sample(self, x, z_prior, GEN):
        # Sample from prior
        z_k = z_prior
        step=0
        while step < self.K:
            g_k = GEN(z_k)
            
            # x = g(z) + eps
            x_k = g_k + torch.randn_like(x, device=self.device) 
            z_k = z_k + self.s * -2*(x_k - g_k) + (torch.sqrt(2*self.s)*torch.randn_like(z_k, device=self.device))
            step += 1
        
        return z_k

In [ ]:
def loss_function(x, z, ):
    log_prior = -0.5 * torch.square(z)
    log_likelihood = -0.5 * torch.square(x - GEN(z))
    return torch.mean(log_prior + log_likelihood)

In [ ]:
NUM_EPOCHS = 1000
BATCH_SIZE = 128
LR = 1e-3
K_prior = 20
K_posterior = 20

device = "cuda" if torch.cuda.is_available() else "cpu"
EBMnet = EBM(784, 784).to(device)
prior_smp = PriorSampler(K_prior, 0.1, device)
post_smp = PosteriorSamples(K_posterior, 0.1, device)
GENnet = Generator(784, 784).to(device)

# Transforms to apply to dataset. Normalising improves data convergence, numerical stability, and regularisation.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create instance of MNIST dataset
dataset = datasets.MNIST(root="dataset/", transform=transform, download=True)

# Create DataLoader to effectively load data from the above dataset in batches
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

loss_fn = loss_function
optimiserEMB = torch.optim.Adam(EBMnet.parameters(), lr=LR, amsgrad=True)
optimiserGEN = torch.optim.Adam(GENnet.parameters(), lr=LR, amsgrad=True)

# Calculate the number of rows and columns for subplots
sample_every = 250 
num_plots = (NUM_EPOCHS // sample_every) - 1
num_cols = min(5, num_plots)  # Maximum of 2 columns
num_rows = (num_plots - 1) // num_cols + 1

for epoch in tqdm(range(NUM_EPOCHS)):
  for batch_idx, (x, _) in enumerate(loader): # Load real data in batches
    x = x.view(-1, 784).to(device)
    batch_size = x.shape[0]

    # 1. Forward Pass -- Sample from exponentially-tilted prior
    z_0 = prior_smp.get_sample(x, EBMnet)
    z_K =  post_smp.get_sample(x, z_0, GENnet)

    # 2. Compute loss
    loss = loss_fn(x, z_K, GENnet)

    # 3. Backward Pass
    optimiserEMB.zero_grad()
    optimiserGEN.zero_grad()
    loss.backward()

    # 4. Update model
    optimiserEMB.step()
    optimiserGEN.step()
  
  if epoch % sample_every == 0 or epoch == NUM_EPOCHS:
    with torch.no_grad():
      z = torch.randn(784, 1).to(device)
      samples = GENnet(z).view(-1, 28, 28).detach().cpu()

      plot_num = (epoch // sample_every) - 1
      row = plot_num // num_cols
      col = (plot_num % num_cols)

      # Plot samples
      axs[row, col].imshow(make_grid(samples, nrow=10).permute(1, 2, 0))
      axs[row, col].set_xticks([])
      axs[row, col].set_yticks([])
      axs[row, col].set_title(f'Epoch: {epoch}')

plt.show()
      




  0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not float